In [2]:
import numpy as np
from functools import partial
import pickle
import matplotlib.pyplot as plt
from sympy.physics.quantum.cg import CG
import tensorflow as tf

In [1]:
import numpy as np
from sympy.physics.quantum.cg import CG

def cgc_multiplier_ccpvdz():
    # x, y: shape (14,)
    
    lm_pairs = [
        [0, 0],
        [0, 0],
        [0, 0],
        [1, -1],
        [1, 0],
        [1, 1],
        [1, -1],
        [1, 0],
        [1, 1],
        [2, -2],
        [2, -1],
        [2, 0],
        [2, 1],
        [2, 2]
    ]

    def cg_tensor(i, j, k):
        l1, m1 = lm_pairs[i]
        l2, m2 = lm_pairs[k]
        l3, m3 = lm_pairs[j]

        return CG(l1, m1, l2, m2, l3, m3).doit()
    
    tens = np.fromfunction(np.vectorize(cg_tensor), (14, 14, 14), dtype=np.int32)

    return tens

In [33]:
W = cgc_multiplier().astype(float)
a1 = np.random.random((14,))
a2 = np.random.random((14,))
a3 = np.random.random((14,))

W[:, 4, :] - W[:, 4, :].T

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        -1.41421356,  0.        ,  0.        , -1.41421356,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.  

In [37]:
tens = np.random.random((3, 3, 3, 3, 3, 3))
vec1 = np.random.random((3, 3))
vec2 = vec1
np.einsum("abcdef,cd,ef->ab", tens, vec1, vec2)

array([[13.39673551, 15.76149442, 15.51063534],
       [14.83188433, 15.01254834, 13.77247093],
       [15.72878687, 14.55829957, 15.12686288]])

In [8]:
def rdm_padding(rdm, Z):
    num_orbitals = np.array([5 if z < 3 else 14 for z in Z])
    rdm_atom_start_indices = np.concatenate([[0], num_orbitals]).cumsum()
    insert_start_positions = rdm_atom_start_indices[1:][rdm_atom_start_indices[1:] - rdm_atom_start_indices[:-1] == 5]
    insertum = np.zeros((9, rdm.shape[1]))
    for index in reversed(insert_start_positions):
        rdm = np.insert(rdm, index, insertum, axis=0)
    rdm = rdm.T
    insertum = np.zeros((9, rdm.shape[1]))
    for index in reversed(insert_start_positions):
        rdm = np.insert(rdm, index, insertum, axis=0)
    for i in np.concatenate([[0], (14 * np.ones_like(Z, dtype=np.int32))[:-1]]).cumsum()[Z <= 2] + 2:
        rdm[:, [i, i + 1, i + 2, i + 3]] = rdm[:, [i + 3, i, i + 1, i + 2]]
        rdm[[i, i + 1, i + 2, i + 3], :] = rdm[[i + 3, i, i + 1, i + 2], :]
    return rdm

In [16]:
def C_padding(C, Z):
    num_orbitals = np.array([5 if z < 3 else 14 for z in Z])
    C_atom_start_indices = np.concatenate([[0], num_orbitals]).cumsum()
    insert_start_positions = C_atom_start_indices[1:][C_atom_start_indices[1:] - C_atom_start_indices[:-1] == 5]
    insertum = np.zeros((9, C.shape[1]))
    for index in reversed(insert_start_positions):
        C = np.insert(C, index, insertum, axis=0)
    return C

In [25]:
def truncate_C(C, Z, R, no_basis_fct=14, epsilon=0.005):
    # out: mo_coeffs (None, 14); mo_coeff_atom_idx (None,); mo_R (None, 3)
    no_orb = len(C)
    R_repeat = np.repeat(R, repeats=no_basis_fct, axis=0)
    C = C_padding(C, Z)
    mo_coeffs = []
    mo_coeffs_Z = []
    mo_coeffs_n = []
    mo_coeffs_R = []
    for i in range(no_orb):
        mo_coeffs_i = C[:, i]
        mo_coeffs_new = []
        mo_coeffs_i_Z = []
        for atom in range(len(mo_coeffs_i) // no_basis_fct):
            mo_coeffs_per_atom = mo_coeffs_i[no_basis_fct * atom : no_basis_fct * (atom + 1)]
            if np.abs(mo_coeffs_per_atom).sum() / np.abs(mo_coeffs_i).sum() > epsilon:
                mo_coeffs_new += list(mo_coeffs_per_atom)
                mo_coeffs_i_Z.append(Z[atom])
        mo_coeffs += mo_coeffs_new
        mo_coeffs_Z += mo_coeffs_i_Z
        mo_coeffs_n.append(len(mo_coeffs_i_Z))

        R_avg = np.abs(mo_coeffs_i)[:, None] * R_repeat / np.abs(mo_coeffs_i).sum()
        mo_coeffs_R += list(R_avg)

    mo_coeffs = np.array(mo_coeffs)
    mo_coeffs_Z = np.array(mo_coeffs_Z)
    mo_coeffs_n = np.array(mo_coeffs_n)
    mo_coeffs_R = np.array(mo_coeffs_R)

    return mo_coeffs, mo_coeffs_Z, mo_coeffs_n, mo_coeffs_R

In [1]:
def mo_neighbours(C, Z, R, e, cutoff):
    has_5_func = Z <= 2
    repeats = 5 * has_5_func + 14 * (1 - has_5_func)
    R_repeat = np.repeat(R, repeats=repeats, axis=0)
    mo_positions = C[:, :, None] * R_repeat[:, None, 3] * np.abs(C) / (np.abs(C).sum(axis=0)[None, :, :])
    mo_positions = mo_positions.sum(axis=0)
    distance_matrix = np.linalg.norm(mo_positions[:, None, :] - mo_positions[None, :, :], axis=-1)
    return distance_matrix

In [4]:
def add_Z_and_N(dict, Z):
    n_mols = len(dict["R"])
    N = Z.shape[0] * np.ones((n_mols,), dtype=np.int32)
    dict["Z"] = np.tile(Z, n_mols)
    dict["N"] = np.tile(N, n_mols)
    return dict

**H2**

In [5]:
# data = np.load("../../data/md_h2.npz", allow_pickle=True)
# Z = np.array([1, 1])
# data = add_Z_and_N(data, Z)
# rdm_hf = data["hf_rdm"]
# h2_rdm_pad = partial(rdm_padding, Z=Z)
# rdm_hf = np.vectorize(h2_rdm_pad, signature="(a,a)->(b,b)")(rdm_hf)
# print(rdm_hf.shape)
# rdm_hf = np.vectorize(rdm_reshape, signature="(a,a)->(c,b,b)")(rdm_hf)
# rdm_hf = np.reshape(rdm_hf, (rdm_hf.shape[0] * rdm_hf.shape[1], rdm_hf.shape[2], rdm_hf.shape[3]))
# data["hf_rdm"] = rdm_hf
# data["N_rdm"] = data["N"] ** 2
# # restrict train coords to the useful ones
# for key in ["train_coords", "hf_train_density", "mp_train_density"]:
#     data[key] = data[key][:, :140]

In [7]:
#with open("../../data/md_h2.npz", "wb") as f:
#     pickle.dump(data, f)

**Benzene**

In [ ]:
data = np.load("../../data/md_benzene_mod.npz", allow_pickle=True)
Z = np.array([6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1])
data = add_Z_and_N(data, Z)
rdm_hf = data["hf_rdm"]
h2_rdm_pad = partial(rdm_padding, Z=Z)
rdm_hf = np.vectorize(h2_rdm_pad, signature="(a,a)->(b,b)")(rdm_hf)
rdm_hf = np.vectorize(rdm_reshape, signature="(a,a)->(c,b,b)")(rdm_hf)
rdm_hf = np.reshape(rdm_hf, (rdm_hf.shape[0] * rdm_hf.shape[1], rdm_hf.shape[2], rdm_hf.shape[3]))
data["hf_rdm"] = rdm_hf
data["N_rdm"] = data["N"] ** 2

In [ ]:
with open("../../data/md_benzene_mod.npz", "wb") as f:
    pickle.dump(data, f)

**Formamide**

In [9]:
data = np.load("../../data/formamide.npz", allow_pickle=True)
Z = np.array([7, 1, 1, 6, 8, 1])
data = add_Z_and_N(data, Z)

# Hartree-Fock RDM
rdm_hf = data["hf_1rdms"]
rdm_pad = partial(rdm_padding, Z=Z)
rdm_hf = np.vectorize(rdm_pad, signature="(a,a)->(b,b)")(rdm_hf)
rdm_hf = np.vectorize(rdm_reshape, signature="(a,a)->(c,b,b)")(rdm_hf)
rdm_hf = np.reshape(rdm_hf, (rdm_hf.shape[0] * rdm_hf.shape[1], rdm_hf.shape[2], rdm_hf.shape[3]))
data["hf_1rdms"] = rdm_hf

# MP2 RDM
rdm_mp = data["mp_1rdms"]
rdm_pad = partial(rdm_padding, Z=Z)
rdm_mp = np.vectorize(rdm_pad, signature="(a,a)->(b,b)")(rdm_mp)
rdm_mp = np.vectorize(rdm_reshape, signature="(a,a)->(c,b,b)")(rdm_mp)
rdm_mp = np.reshape(rdm_mp, (rdm_mp.shape[0] * rdm_mp.shape[1], rdm_mp.shape[2], rdm_mp.shape[3]))
data["mp_1rdms"] = rdm_mp

data["N_rdm"] = data["N"] ** 2

In [10]:
with open("../../data/formamide.npz", "wb") as f:
    pickle.dump(data, f)

In [11]:
data = np.load("../../data/formamide.npz", allow_pickle=True)
data.keys()

dict_keys(['R', 'grid_coordss', 'sphere_coordss', 'hf_1rdms', 'mp_1rdms', 'hf_grid_densities', 'hf_sphere_densities', 'mp_grid_densities', 'mp_sphere_densities', 'dft_grid_densities', 'dft_sphere_densities', 'Z', 'N', 'N_rdm'])

In [12]:
data["mp_1rdms"].shape

(33552, 14, 14)